# 상위 100개 샴푸 브랜드의 상품명 추출

In [124]:
import pandas as pd
list_brand=pd.read_excel('shampoo_top100_list.xlsx',encoding='utf-8')

In [125]:
list_brand=list_brand['brand']
list_brand=set(list_brand)
# print(list_brand)
# len(list_brand)

In [127]:
brand=[]
for i in list_brand:
    brand.append(i)
# brand
brand_s=sorted(brand)
print(brand_s)

In [2]:
from selenium import webdriver  # 라이브러리(모듈) 가져오라
from selenium.webdriver import ActionChains as AC
from tqdm import tqdm
from tqdm import tqdm_notebook
import re
from time import sleep
import time
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from tqdm import tqdm
from tqdm import tqdm_notebook
from selenium.webdriver.common.keys import Keys

In [ ]:
# beautifulsoup버전
# url = 'http://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000400080001'

# html = requests.get(url)
# soup = BeautifulSoup(html.content, 'lxml')

# t_list = soup.select('#Contents > ul > li.flag > div > a')
# for each in t_list:
#     print(each.attrs['data-ref-goodsno'])

In [1]:
from selenium import webdriver
driver = webdriver.Chrome('chromedriver.exe')

url_list = []
for j in range(8):
    url=('http://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000400080001&fltDispCatNo=&prdSort=01&pageIdx={}&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=Y&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&searchOnlBrndCdArr=A001441&searchOnlBrndCdArr=A002789&searchOnlBrndCdArr=A001218&searchOnlBrndCdArr=A001328&searchOnlBrndCdArr=A000406&searchOnlBrndCdArr=A000084&searchOnlBrndCdArr=A002513&searchOnlBrndCdArr=A000200&searchOnlBrndCdArr=A000124&searchOnlBrndCdArr=A000709&searchOnlBrndCdArr=A000692&searchOnlBrndCdArr=A000134&searchOnlBrndCdArr=A000036&searchOnlBrndCdArr=A000754&searchOnlBrndCdArr=A002694&searchOnlBrndCdArr=A000929&searchOnlBrndCdArr=A001536&searchOnlBrndCdArr=A000559&searchOnlBrndCdArr=A001444&searchOnlBrndCdArr=A002192&searchOnlBrndCdArr=A001616&searchOnlBrndCdArr=A000048&searchOnlBrndCdArr=A001461&searchOnlBrndCdArr=A000405&searchOnlBrndCdArr=A002482'.format(j))
    driver.get(url)
    products = driver.find_elements_by_css_selector('div.prd_info a')
    for product in products:
            url = product.get_attribute('data-ref-goodsno')   
            url_list.append(url)
url_list=list(set(url_list))
url_list
len(url_list)

144

In [2]:
url_list=list(set(url_list))

# 각 상품의 리뷰가져오기

In [136]:
###new ver

import json
import requests

#url = 'http://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo={}&dispCatNo=1000001000400080001'.format('A000000128661')


for url in url_list:
    mylist=[]
    review=[]
    title=[]
    product=[]
    detail1=[]
    detail2=[]
    detail3=[]
    detail4=[]

    for j in range(100):
        try:
            rev_url = ('http://www.oliveyoung.co.kr/store/goods/getGdasNewListJson.do?goodsNo={0}&gdasSort=02&itemNo=all_search&pageIdx={1}&colData=&cTypeLength=0'.format(url,j))
            html = requests.get(rev_url)
            rev_json = html.json()
            # print(rev_json)
            for i in range(10):

                a=rev_json['gdasList'][i]['gdasCont']
                b=rev_json['gdasList'][i]['shrtGdasCont']
                c=rev_json['gdasList'][i]['goodsNm']

                try:
                    d1=rev_json['gdasList'][i]['evalList'][0]['evalAnsCont']
                    d2=rev_json['gdasList'][i]['evalList'][1]['evalAnsCont']
                    d3=rev_json['gdasList'][i]['evalList'][2]['evalAnsCont']
                    d4=rev_json['gdasList'][i]['evalList'][3]['evalAnsCont']
                except:
                    d1=" "
                    d2=" "
                    d3=" "
                    d4=" "

                review.append(a)
                title.append(b)
                product.append(c)
                detail1.append(d1)
                detail2.append(d2)
                detail3.append(d3)
                detail4.append(d4)
        except:
            pass

    mylist=pd.DataFrame({'title':title,'review':review,'product':product,'detail1':detail1,'detail2':detail2,'detail3':detail3,'detail4':detail4})
    mylist.to_excel('{}.xlsx'.format(url),encoding='utf-8')

# 각 상품의 성분 가져오기

In [ ]:
mylist=[]
list_con=[]
for url in url_list:
    url=('http://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo={}&dispCatNo=1000001000400080001'.format(url))
    driver = webdriver.Chrome('chromedriver.exe')
    driver.get(url)
    time.sleep(5)
    
    driver.find_element_by_xpath("//*[@id='buyInfo']/a").send_keys(Keys.ENTER)
    time.sleep(10)
    
    con=driver.find_element_by_css_selector('#artcInfo > dl:nth-child(7) > dd')
    con=con.text
    driver.close()
    list_con.append(con)

In [91]:
import pandas as pd
mylist=pd.DataFrame({'title':url_list,'detail':list_con})
mylist.to_excel('detail.xlsx',encoding='utf-8')

In [104]:
data=pd.read_excel('detail2.xlsx')
data.head()

,Unnamed: 0,title,detail
0,0,A000000116670,"정제수,소듐메틸코코일타우레이트,라우릴베타인,소듐메틸올레오일타우레이트,소듐코코일이세티..."
1,1,A000000112800,"정제수,소듐라우레스설페이트,글리세린,코카미도프로필베타인,코코-글루코사이드,글라이콜다..."
2,2,A000000014522,"소듐메틸코코일타우레이트,티트리잎추출물,박하추출물,소엽잎추출물,소듐코코암포아세테이트,..."
3,3,A000000120983,"정제수,소듐라우레스설페이트,소듐라우릴설페이트,어성초추출물,코카마이드디이에이,\n코카..."
4,4,A000000128193,"이브로쉐퓨리파잉샴푸300ML:정제수,암모늄라우릴설페이트,데실글루코사이드,코카미도프로..."


In [105]:
split=data.detail.str.split(',')
spit=sorted(split)
split=split.apply(lambda x: pd.Series(x))
split

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,정제수,소듐메틸코코일타우레이트,라우릴베타인,소듐메틸올레오일타우레이트,소듐코코일이세티오네이트,소듐클로라이드,에리스리톨,판테놀,칼슘판토테네이트,바이오틴,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,정제수,소듐라우레스설페이트,글리세린,코카미도프로필베타인,코코-글루코사이드,글라이콜다이스테아레이트,세틸알코올,소듐라우릴설페이트,향료,라우릭애씨드,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,소듐메틸코코일타우레이트,티트리잎추출물,박하추출물,소엽잎추출물,소듐코코암포아세테이트,한련초추출물,글리세린,\n소듐코코일이세티오네이트,라우릴글루코사이드,프로판다이올,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,정제수,소듐라우레스설페이트,소듐라우릴설페이트,어성초추출물,코카마이드디이에이,\n코카미도프로필베타인,부틸렌글라이콜,징크피리치온,판테놀,나이아신아마이드,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,이브로쉐퓨리파잉샴푸300ML:정제수,암모늄라우릴설페이트,데실글루코사이드,코카미도프로필베타인,글리세릴올리에이트,코코-글루코사이드,징크글루코네이트,소듐벤조에이트,향료,글리세린,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,부탄,이소부탄,프로판,쌀전분,변성알코올,향료,리모넨,디스테아릴디모늄클로라이드,리날룰,제라니올,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,정제수,소듐라우레스설페이트,코카마이드미파,라우릴베타인,에탄올,바이오틴(0.0005%),청색1호,카프릴릴글라이콜,카라멜,신코나껍질추출물,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,정제수,소듐라우레스설페이트,암모늄라우릴설페이트,의이인추출물,감초추출물,구주소나무잎추출물,녹차추출물,동백오일,맥아추출물,생강추출물,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,정제수,소듐라우레스설페이트,소듐라우릴설페이트,글라이콜다이스테아레이트,코카미도프로필베타인,다이메티콘,에리스리톨,세틸알코올,글리세릴올리에이트,향료,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,정제수,다이소듐라우레스설포석시네이트,라우릴하이드록시설테인,코카마이드메틸엠이에이,스타이렌/아크릴레이트코폴리머,글리세린,소듐코코일이세티오네이트,바이오틴,세라마이드엔피,사발팜열매추출물,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
data2=pd.concat([data['title'],split],axis=1,ignore_index=True)

In [107]:
data2

,0,1,2,3,4,5,6,7,8,9,...,105,106,107,108,109,110,111,112,113,114
0,A000000116670,정제수,소듐메틸코코일타우레이트,라우릴베타인,소듐메틸올레오일타우레이트,소듐코코일이세티오네이트,소듐클로라이드,에리스리톨,판테놀,칼슘판토테네이트,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A000000112800,정제수,소듐라우레스설페이트,글리세린,코카미도프로필베타인,코코-글루코사이드,글라이콜다이스테아레이트,세틸알코올,소듐라우릴설페이트,향료,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A000000014522,소듐메틸코코일타우레이트,티트리잎추출물,박하추출물,소엽잎추출물,소듐코코암포아세테이트,한련초추출물,글리세린,\n소듐코코일이세티오네이트,라우릴글루코사이드,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A000000120983,정제수,소듐라우레스설페이트,소듐라우릴설페이트,어성초추출물,코카마이드디이에이,\n코카미도프로필베타인,부틸렌글라이콜,징크피리치온,판테놀,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A000000128193,이브로쉐퓨리파잉샴푸300ML:정제수,암모늄라우릴설페이트,데실글루코사이드,코카미도프로필베타인,글리세릴올리에이트,코코-글루코사이드,징크글루코네이트,소듐벤조에이트,향료,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,A000000009621,부탄,이소부탄,프로판,쌀전분,변성알코올,향료,리모넨,디스테아릴디모늄클로라이드,리날룰,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,A000000013863,정제수,소듐라우레스설페이트,코카마이드미파,라우릴베타인,에탄올,바이오틴(0.0005%),청색1호,카프릴릴글라이콜,카라멜,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,A000000120878,정제수,소듐라우레스설페이트,암모늄라우릴설페이트,의이인추출물,감초추출물,구주소나무잎추출물,녹차추출물,동백오일,맥아추출물,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,A000000127765,정제수,소듐라우레스설페이트,소듐라우릴설페이트,글라이콜다이스테아레이트,코카미도프로필베타인,다이메티콘,에리스리톨,세틸알코올,글리세릴올리에이트,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,A000000117235,정제수,다이소듐라우레스설포석시네이트,라우릴하이드록시설테인,코카마이드메틸엠이에이,스타이렌/아크릴레이트코폴리머,글리세린,소듐코코일이세티오네이트,바이오틴,세라마이드엔피,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
data2.to_excel('detail_list.xlsx')